<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:


      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66


    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 14:42:23
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: -10.31 K (-0.07%)
Current PnL: -16.86 L (-11.34%)
CY Booked + Current PnL: -5.42 L (-3.64%)
-------------------
Total profit:  2.04 L
Total loss:  -18.89 L
-------------------
Total Booked + Current PnL: 21.56 L (17.72%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.37 L (57.0%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.87%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.61,65.0,H-MC,3.49,196200.0,31530.0,11792.0,0.05,19.15,6.01,26.31,89.0,2.67,1.44,28.66,X40N,NTT,AC
76,TTKPRESTIG,770.00,102.77,52.0,M-SC,3.71,87344.0,-13433.0,13530.0,-1.02,-13.33,15.49,0.09,245.0,-0.99,0.64,14.75,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.72,60.0,H-MC,2.69,213300.0,21558.0,16189.0,-0.29,11.24,7.59,19.69,99.0,1.33,1.56,18.80,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,64.0,H-MC,7.92,111639.0,11135.0,17974.0,-0.31,11.08,16.10,28.96,79.0,0.62,0.82,48.81,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,58.0,H-LC,4.96,220241.0,9875.0,18897.0,-0.23,4.69,8.58,13.68,37.0,0.52,1.61,21.99,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,TANLA,1963.11,-15.81,77.0,H-SC,12.38,225904.0,-44857.0,374797.0,5.52,-16.57,165.91,121.86,133.0,-0.12,1.65,76.72,AR,ATH,IT
78,UNIONBANK,163.00,-11.63,67.0,M-LC,7.06,159476.0,18636.0,25851.0,2.69,13.23,16.21,31.59,66.0,0.72,1.17,43.22,XY24,NTT,BANKS
60,SAMMAANCAP,326.00,-173.86,69.0,M-SC,3.30,85350.0,-16860.0,110247.0,2.20,-16.50,129.17,91.37,208.0,-0.15,0.62,38.55,XY25,NTT,FINANCE
59,SAIL,228.00,48.36,70.0,M-MC,7.56,234175.0,9213.0,158443.0,2.11,4.10,67.66,74.53,192.0,0.06,1.71,37.67,XY24,BTT,STEEL
26,GREENPANEL,537.00,234.63,62.0,M-SC,1.79,149328.0,-33750.0,112728.0,1.78,-18.43,75.49,43.14,230.0,-0.30,1.09,40.46,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,QUESS,986.00,-41.60,43.0,M-SC,39.88,58035.0,-6971.0,157902.0,-2.79,-10.72,272.08,232.18,198.0,-0.04,0.42,1.07,XY24,NTT,MISC
80,VAIBHAVGBL,521.00,61.43,64.0,H-SC,6.64,140098.0,-42677.0,153744.0,-2.48,-23.35,109.74,60.77,125.0,-0.28,1.03,25.48,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3797.30,59.0,M-SC,5.38,121575.0,-25793.0,77613.0,-2.46,-17.50,63.84,35.16,236.0,-0.33,0.89,24.18,XY24,NTT,AUTO
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130862.0,7526.0,126243.0,-2.14,6.10,96.47,108.45,119.0,0.06,0.96,29.82,AR,ATH,MISC
55,RELAXO,1176.00,-33.00,52.0,H-SC,1.91,91321.0,-53839.0,129767.0,-1.95,-37.09,142.10,52.31,136.0,-0.41,0.67,22.24,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.23,57.0,H-SC,16.92,201922.0,1968.0,95267.0,0.13,0.98,47.18,48.63,115.0,0.02,1.48,12.65,XR,NTT,MISC
34,ICICIPRULI,790.00,-20.54,46.0,H-MC,2.18,136730.0,942.0,41812.0,0.00,0.69,30.58,31.48,107.0,0.02,1.00,12.90,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.34,59.0,H-MC,1.92,248882.0,1151.0,73918.0,0.07,0.46,29.70,30.30,92.0,0.02,1.82,13.82,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.81,52.0,H-MC,4.09,200310.0,-510.0,69287.0,0.01,-0.25,34.59,34.25,80.0,-0.01,1.47,4.93,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,102.77,52.0,M-SC,3.71,87344.0,-13433.0,13530.0,-1.02,-13.33,15.49,0.09,245.0,-0.99,0.64,14.75,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,94.87,54.0,H-SC,9.41,123093.0,-19386.0,38910.0,0.18,-13.61,31.61,13.70,163.0,-0.50,0.90,45.28,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,62.0,H-SC,1.26,226800.0,-42867.0,79199.0,-0.49,-15.90,34.92,13.47,138.0,-0.54,1.66,14.71,XY24,NTT,PAINTS
18,CERA,9475.0,-20.29,44.0,H-SC,2.11,145280.0,-30623.0,72640.0,-0.49,-17.41,50.00,23.89,149.0,-0.42,1.06,25.47,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.68,53.0,H-MC,7.44,108614.0,-23221.0,65451.0,-0.30,-17.61,60.26,32.03,98.0,-0.35,0.80,21.66,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,48.36,70.0,M-MC,7.56,234175.0,9213.0,158443.0,2.11,4.10,67.66,74.53,192.0,0.06,1.71,37.67,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.61,65.0,H-MC,3.49,196200.0,31530.0,11792.0,0.05,19.15,6.01,26.31,89.0,2.67,1.44,28.66,X40N,NTT,AC
83,VOLTAS,1530.00,1.72,60.0,H-MC,2.69,213300.0,21558.0,16189.0,-0.29,11.24,7.59,19.69,99.0,1.33,1.56,18.80,XY25,NTT,AC
17,CAMS,5211.76,0.60,67.0,H-SC,3.46,114492.0,12488.0,31440.0,0.34,12.24,27.46,43.06,122.0,0.40,0.84,31.45,X40N,ATH,MISC
29,HAVELLS,2069.16,0.34,59.0,H-MC,1.92,248882.0,1151.0,73918.0,0.07,0.46,29.70,30.30,92.0,0.02,1.82,13.82,X40,ATH,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,64.0,H-MC,7.92,111639.0,11135.0,17974.0,-0.31,11.08,16.10,28.96,79.0,0.62,0.82,48.81,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.79,51.0,M-SC,0.65,100178.0,8680.0,70075.0,0.02,9.49,69.95,86.07,223.0,0.12,0.73,45.48,XR,NTT,DURABLES
85,WIPRO,420.00,-10.16,57.0,M-LC,5.66,158156.0,7211.0,102248.0,-0.72,4.78,64.65,72.51,53.0,0.07,1.16,11.09,XR,NTT,IT
36,INDIAMART,4850.92,-53.15,34.0,H-SC,11.48,130862.0,7526.0,126243.0,-2.14,6.10,96.47,108.45,119.0,0.06,0.96,29.82,AR,ATH,MISC
11,BANKINDIA,190.00,-27.95,66.0,H-MC,6.33,185006.0,5198.0,107211.0,1.08,2.89,57.95,62.52,88.0,0.05,1.35,37.96,XR,NTT,BANKS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302763.0,-13079.0,125737.0,0.21,-4.14,41.53,35.67,5.0,-0.10,2.22,9.14,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-12.93,48.0,H-LC,7.78,219727.0,-32041.0,85254.0,0.73,-12.73,38.80,21.13,27.0,-0.38,1.61,18.19,X40,ATH,PAINTS
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200466.0,328.0,20568.0,-0.39,0.16,10.26,10.44,4.0,0.02,1.47,5.74,X40,NTT,FMCG
30,HCLTECH,1943.91,1.08,49.0,H-LC,9.74,230963.0,-10961.0,74232.0,-1.52,-4.53,32.14,26.15,8.0,-0.15,1.69,9.14,X40,ATH,IT
43,JIOFIN,387.00,0.06,51.0,H-LC,13.14,271932.0,4567.0,60886.0,-0.36,1.71,22.39,24.48,48.0,0.08,1.99,57.62,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-0.29,68.0,H-LC,1.47,247545.0,1394.0,77803.0,-0.43,0.57,31.43,32.18,16.0,0.02,1.81,36.81,X200,ATH,IT
41,ITC,452.0,-37.92,49.0,H-LC,2.38,200466.0,328.0,20568.0,-0.39,0.16,10.26,10.44,4.0,0.02,1.47,5.74,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.11,53.0,H-LC,4.18,279630.0,14204.0,42588.0,-1.17,5.35,15.23,21.40,11.0,0.33,2.05,13.07,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,58.0,H-LC,4.96,220241.0,9875.0,18897.0,-0.23,4.69,8.58,13.68,37.0,0.52,1.61,21.99,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.44,54.0,H-LC,5.74,215200.0,10700.0,30709.0,-0.47,5.23,14.27,20.25,10.0,0.35,1.58,14.42,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNITDSPR,1644.00,-12.94,52.0,H-LC,7.77,228313.0,-5831.0,54453.0,0.00,-2.49,23.85,20.77,86.0,-0.11,1.67,4.26,X40N,NTT,BREWERIES
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,297040.0,-48916.0,123777.0,-0.53,-14.14,41.67,21.64,1.0,-0.40,2.17,5.23,X40,ATH,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200466.0,328.0,20568.0,-0.39,0.16,10.26,10.44,4.0,0.02,1.47,5.74,X40,NTT,FMCG
30,HCLTECH,1943.91,1.08,49.0,H-LC,9.74,230963.0,-10961.0,74232.0,-1.52,-4.53,32.14,26.15,8.0,-0.15,1.69,9.14,X40,ATH,IT
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302763.0,-13079.0,125737.0,0.21,-4.14,41.53,35.67,5.0,-0.10,2.22,9.14,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,7969.85,3.38,64.0,H-LC,26.99,166030.0,-20065.0,232459.0,0.08,-10.78,140.01,114.13,15.0,-0.09,1.22,22.27,AR,ATH,ELECTRICAL
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200466.0,328.0,20568.0,-0.39,0.16,10.26,10.44,4.0,0.02,1.47,5.74,X40,NTT,FMCG
19,CIPLA,1795.00,-19.44,54.0,H-LC,5.74,215200.0,10700.0,30709.0,-0.47,5.23,14.27,20.25,10.0,0.35,1.58,14.42,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-12.93,48.0,H-LC,7.78,219727.0,-32041.0,85254.0,0.73,-12.73,38.80,21.13,27.0,-0.38,1.61,18.19,X40,ATH,PAINTS
56,RELIANCE,1533.00,-12.16,58.0,H-LC,4.96,220241.0,9875.0,18897.0,-0.23,4.69,8.58,13.68,37.0,0.52,1.61,21.99,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.47,57.0,H-LC,12.30,297040.0,-48916.0,123777.0,-0.53,-14.14,41.67,21.64,1.0,-0.40,2.17,5.23,X40,ATH,IT
39,INFY,2275.00,-16.37,55.0,H-LC,8.18,325251.0,12363.0,159308.0,-0.88,3.95,48.98,54.87,3.0,0.08,2.38,10.86,X40,BTT,IT
41,ITC,452.00,-37.92,49.0,H-LC,2.38,200466.0,328.0,20568.0,-0.39,0.16,10.26,10.44,4.0,0.02,1.47,5.74,X40,NTT,FMCG
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302763.0,-13079.0,125737.0,0.21,-4.14,41.53,35.67,5.0,-0.10,2.22,9.14,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,9.86,260904.0,-715.0,119938.0,-0.01,-0.27,45.97,45.57,7.0,-0.01,1.91,11.02,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7016.67,65.0,L-SC,5.20,78111.0,-15699.0,92319.0,0.71,-16.73,118.19,81.67,269.0,-0.17,0.57,51.26,XR,NTT,CERAMICS
21,COFFEEDAY,80.00,-43.66,51.0,L-SC,42.23,84576.0,-28973.0,69022.0,0.20,-25.52,81.61,35.27,268.0,-0.42,0.62,107.00,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-173.86,69.0,M-SC,3.30,85350.0,-16860.0,110247.0,2.20,-16.50,129.17,91.37,208.0,-0.15,0.62,38.55,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-4.67,47.0,H-SC,12.49,88713.0,-12250.0,106784.0,-0.59,-12.13,120.37,93.63,148.0,-0.11,0.65,32.14,SR,ATH,CHEMICALS
48,MASFIN,398.61,-16.89,52.0,H-SC,12.99,94530.0,-3450.0,25050.0,0.32,-3.52,26.50,22.05,152.0,-0.14,0.69,37.41,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1113.72,70.0,M-SC,4.61,164989.0,-16154.0,61623.0,1.17,-8.92,37.35,25.10,235.0,-0.26,1.21,32.22,XY24,NTT,HEALTHCARE
70,TANLA,1963.11,-15.81,77.0,H-SC,12.38,225904.0,-44857.0,374797.0,5.52,-16.57,165.91,121.86,133.0,-0.12,1.65,76.72,AR,ATH,IT
60,SAMMAANCAP,326.00,-173.86,69.0,M-SC,3.30,85350.0,-16860.0,110247.0,2.20,-16.50,129.17,91.37,208.0,-0.15,0.62,38.55,XY25,NTT,FINANCE
74,TITAGARH,1548.00,0.96,68.0,H-SC,3.16,165690.0,-26325.0,105213.0,0.03,-13.71,63.50,41.08,158.0,-0.25,1.21,40.40,XY24,NTT,ENGINEERING
7,ATULAUTO,844.00,3797.30,59.0,M-SC,5.38,121575.0,-25793.0,77613.0,-2.46,-17.50,63.84,35.16,236.0,-0.33,0.89,24.18,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.08
1,25,43.99
2,50,74.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.53
LC    32.11
MC    23.35
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.35
X40      15.37
X40N     11.97
XR       11.78
AR        8.82
XY25      8.25
OX40N     8.21
X200      1.81
MH        1.70
X5K       1.44
SR        1.29
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.32
H-LC    25.38
H-MC    20.33
M-SC    13.63
M-LC     5.66
M-MC     2.71
L-SC     1.58
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.68,-12.44,73.75
FMCG,11.92,-4.10,40.52
FINANCE,9.74,-15.55,60.81
MISC,7.43,-13.19,76.11
BANKS,6.37,-14.76,75.13
PAINTS,5.87,-15.23,32.41
ELECTRICAL,5.77,-4.60,62.25
AC,3.72,4.68,11.28
INSURANCE,3.44,-6.98,43.67


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2834218.0,22
XR,1300269.0,14
AR,1292789.0,9
X40,749006.0,10
X40N,571808.0,9
OX40N,526504.0,10
XY25,337131.0,6
SR,245569.0,2
X5K,120598.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3407524.0,29
M-SC,1382126.0,17
H-LC,1226609.0,14
H-MC,1092671.0,15
M-LC,377612.0,4
M-MC,306849.0,2
L-SC,244976.0,3
L-MC,60405.0,1
L-LC,38437.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1025355.0      6
M-SC       XY24       799404.0      7
H-SC       AR         778858.0      5
           XR         754625.0      7
H-MC       XY24       486958.0      4
H-LC       X40        463139.0      5
           AR         352397.0      2
M-MC       XY24       306849.0      2
H-SC       X40N       287533.0      4
           OX40N      252044.0      4
           SR         245569.0      2
H-MC       X40        214739.0      4
H-LC       X40N       210899.0      3
M-SC       AR         161534.0      2
L-SC       XR         161341.0      2
M-LC       XY24       154766.0      2
H-MC       XY25       126962.0      2
M-SC       OX40N      125374.0      4
M-LC       X5K        120598.0      1
M-SC       XR         114439.0      2
           XY25       110247.0      1
H-MC       XR         107211.0      1
M-LC       XR         102248.0      1
L-SC       OX40N       83635.0      1
H-LC       X200        77803.0      1
H-MC       X40N        73376.0      2
M-SC       X40         71128.0      1
H-MC       OX40N       65451.0      1
H-SC       MH          63540.0      1
H-LC       XY25        61485.0      2
           XY24        60886.0      1
L-MC       XR          60405.0      1
L-LC       XY25        38437.0      1
H-MC       MH          17974.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 53.0 seconds
